In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers sentencepiece

Looking in indexes: https://download.pytorch.org/whl/cu121
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 25.6 MB/s  0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "rafalposwiata/deproberta-large-depression"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda")
model.to(device)
model.eval()
print("It's working (i think)")

It's working (i think)


In [16]:
# just an example
text = "I don't think that its possible for this to get any worse."

# tokenize and prepare input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

# Run inference (no gradients i'll just add that later)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()

labels = ["not depressed", "depressed"]

print(f"Input: {text}")
print(f"Prediction: {labels[predicted_class]}")
print(f"Confidence: {probs[0][predicted_class].item():.3f}")


Input: I don't think that its possible for this to get any worse.


IndexError: list index out of range

### Left that bug in there because I found that there are a couple of issues. I think this is happening because it can have 3 labels (depressed, not depressed or unknown)

In [26]:
text = """

And because I had such a wonderful Christmas as home I felt it would be a good last memory for my family  

If I hadn’t called my friend I most likely would have had more than 6 

But he came over and he brought the paramedics with him
""" # I pulled this from a random line in the data from the suicide 2019.

# Tokenize input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()

# Use model’s built-in label map
id2label = model.config.id2label
label_name = id2label[predicted_class]

print(f"Input: {text}")
print(f"Prediction: {label_name}")
print(f"Confidence: {probs[0][predicted_class].item():.3f}")

# this should have all 3


Input: 

And because I had such a wonderful Christmas as home I felt it would be a good last memory for my family  

If I hadn’t called my friend I most likely would have had more than 6 

But he came over and he brought the paramedics with him

Prediction: moderate
Confidence: 0.748
